#### Rating by movie title -> using dataframes on the top of RDD. the latest version of spark support dataframes an is faster than RDD

In [ ]:
import findspark
findspark.init()
findspark.find()
import pyspark
findspark.find()

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as func
from pyspark.sql.types import StructType, StructField, IntegerType, LongType

In [ ]:
spark = SparkSession.builder.appName("PopularMovies").getOrCreate()

In [32]:
def loadMovieNames():
    movieNames = {}
    # CHANGE THIS TO THE PATH TO YOUR u.ITEM FILE:
    with open('C:/Users/afoto001.000/Desktop/Abraham_edu_doc/UvA/Master/Big data/BigData/bigdata/data/ml-latest/movies.csv') as f:
        for line in f:
            fields = line.split(',')
            print( fields[1])


In [33]:
loadMovieNames()

title
Toy Story (1995)
Jumanji (1995)
Grumpier Old Men (1995)
Waiting to Exhale (1995)
Father of the Bride Part II (1995)
Heat (1995)
Sabrina (1995)
Tom and Huck (1995)
Sudden Death (1995)
GoldenEye (1995)
"American President
Dracula: Dead and Loving It (1995)
Balto (1995)
Nixon (1995)
Cutthroat Island (1995)
Casino (1995)
Sense and Sensibility (1995)
Four Rooms (1995)
Ace Ventura: When Nature Calls (1995)
Money Train (1995)
Get Shorty (1995)
Copycat (1995)
Assassins (1995)
Powder (1995)
Leaving Las Vegas (1995)
Othello (1995)
Now and Then (1995)
Persuasion (1995)
"City of Lost Children
Shanghai Triad (Yao a yao yao dao waipo qiao) (1995)
Dangerous Minds (1995)
Twelve Monkeys (a.k.a. 12 Monkeys) (1995)
Wings of Courage (1995)
Babe (1995)
Carrington (1995)
Dead Man Walking (1995)
Across the Sea of Time (1995)
It Takes Two (1995)
Clueless (1995)
"Cry
Richard III (1995)
Dead Presidents (1995)
Restoration (1995)
Mortal Kombat (1995)
To Die For (1995)
How to Make an American Quilt (1995)
Se

UnicodeDecodeError: 'charmap' codec can't decode byte 0x81 in position 4954: character maps to <undefined>